## 1. Import required libraries

In [1]:
# Vanilla GAN with Multi GPUs + Naming Layers using OrderedDict
# Code by GunhoChoi and TaeyoungKim

import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

## 2. Hyperparameter Setting

In [2]:
# Set Hyperparameters
# change num_gpu to the number of gpus you want to use

epoch = 1
batch_size = 512
learning_rate = 0.0001
num_gpus = 1
z_size = 50
middle_size = 200

## 3. Data Setting

In [3]:
# Download Data
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)

# Set Data Loader(input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)


## 4. Generator

In [4]:
# Generator receives random noise z and create 1x28x28 image
# we can name each layer using OrderedDict

class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.layer1 = nn.Sequential(OrderedDict([
                    ('fc1',nn.Linear(z_size, middle_size)),
                    ('bn1',nn.BatchNorm1d(middle_size)),
                    ('act1',nn.ReLU()),
        ]))
        
        self.layer2 = nn.Sequential(OrderedDict([
                    ('fc2',nn.Linear(middle_size,784)),
                    #('bn2',nn.BatchNorm2d(784)),
                    ('bn2',nn.BatchNorm1d(784)),
                    ('tanh', nn.Tanh()),
        ]))
    
    def forward(self,z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = out.view(batch_size//num_gpus,1,28,28)
        #view는 강제로 차원을 변형해주기 위해 쓰는 함수
        
        return out

## 5. Discriminator

In [5]:
# Discriminator receives 1x28x28 image and returns a float number 0~1
# we can name each layer using OrderedDict

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.layer1 = nn.Sequential(OrderedDict([
                        ('fc1',nn.Linear(784, middle_size)),
                        ('bn1',nn.BatchNorm1d(middle_size)),
                        ('act1',nn.LeakyReLU()),
        ]))
        
        self.layer2 = nn.Sequential(OrderedDict([
                        ('fc2',nn.Linear(middle_size,1)),
                        #('bc2',nn.BatchNorm1d(1)),
                        ('act2',nn.Sigmoid()),
        ]))
        
    def forward(self,x):
        out = x.view(batch_size//num_gpus,-1)
        out = self.layer1(out)
        out = self.layer2(out)
        
        return out

## 6. Put instances on Multi-gpu

In [6]:
# Put class objects on Multiple GPUs using 
# torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)
# device_ids: default all devices / output_device: default device 0 
# along with .cuda()

generator = nn.DataParallel(Generator()).cuda()
discriminator = nn.DataParallel(Discriminator()).cuda()

## 7. Check layers

In [7]:
# Get parameter list by using class.state_dict().keys()

gen_params = generator.state_dict().keys()
dis_params = discriminator.state_dict().keys()

for i in gen_params:
    print(i)

module.layer1.fc1.weight
module.layer1.fc1.bias
module.layer1.bn1.weight
module.layer1.bn1.bias
module.layer1.bn1.running_mean
module.layer1.bn1.running_var
module.layer1.bn1.num_batches_tracked
module.layer2.fc2.weight
module.layer2.fc2.bias
module.layer2.bn2.weight
module.layer2.bn2.bias
module.layer2.bn2.running_mean
module.layer2.bn2.running_var
module.layer2.bn2.num_batches_tracked


## 8. Set Loss function & Optimizer

In [8]:
# loss function and optimizers and labels for training

loss_func = nn.MSELoss()
gen_optim = torch.optim.Adam(generator.parameters(), lr = learning_rate, betas = (0.5,0.999))
dis_optim = torch.optim.Adam(discriminator.parameters(), lr = learning_rate, betas = (0.5,0.999))

ones_label = Variable(torch.ones(batch_size,1)).cuda()
zeros_label = Variable(torch.zeros(batch_size,1)).cuda()

## 9. Restore Model

In [9]:
# model restore if any

try :
    generator, discriminator = torch.load('./model/vanilla_gan.pkl')
    print("\n---------------model restored-----------------\n")

except:
    print("\n---------------model not restored-----------------\n")
    pass


---------------model not restored-----------------



## 10. Train model

In [12]:
#train

for i in range(epoch):
    for j,(image,label) in enumerate(train_loader):
        image = Variable(image).cuda()
        
        # discriminator
        
        dis_optim.zero_grad()
        
        z = Variable(init.normal_(torch.Tensor(batch_size,z_size),mean = 0, std = 0.1)).cuda()
        gen_fake = generator.forward(z)
        dis_fake = discriminator.forward(gen_fake)
        
        dis_real = discriminator.forward(image)
        dis_loss = torch.sum(loss_func(dis_fake,zeros_label)) + torch.sum(loss_func(dis_real,ones_label))
        #fake image는 가짜 그림과 loss, 진짜 MNIST image는 실제 그림과 loss
        dis_loss.backward(retain_graph = True)
        dis_optim.step()
        
        # generator
        
        gen_optim.zero_grad()
        
        z = Variable(init.normal_(torch.Tensor(batch_size,z_size),mean = 0, std = 0.1)).cuda()
        gen_fake = generator.forward(z)
        dis_fake = discriminator.forward(gen_fake)
        
        gen_loss = torch.sum(loss_func(dis_fake, ones_label))
        #fake classified as real
        gen_loss.backward()
        gen_optim.step()
        
        #model save
        if j % 100 == 0:
            print(gen_loss, dis_loss)
            #torch.save([generator,discriminator],'./vanilla_gan.pkl') #한번에 모델을 저장! but Warning 표시가 뜬다..
            torch.save({'generator': generator.state_dict()}, './vanilla_generator.pkl')
            torch.save({'discriminator':discriminator.state_dict()}, './vanilla_discriminator.pkl')
            #파일의 경로 신경 써 주기!
            
            print("{}th iteration gen_loss: {} dis_loss: {}".format(i,gen_loss.data,dis_loss.data))
            v_utils.save_image(gen_fake.data[0:25],"./gen_{}_{}.png".format(i,j), nrow=5)
            #생성된 이미지가 저장되는 부분, 파일 경로 확인!

tensor(0.2823, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.4545, device='cuda:0', grad_fn=<AddBackward0>)
0th iteration gen_loss: 0.2823394238948822 dis_loss: 0.45450830459594727
tensor(0.3170, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.4080, device='cuda:0', grad_fn=<AddBackward0>)
0th iteration gen_loss: 0.31698527932167053 dis_loss: 0.4080137014389038
